## Projet de Files d'Attente - MPRO

### François Lamothe & Alexis Reymann

#### 0 - Imports

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import heapq as hp

####  1 - Définition des classes

On définit premièrement les classes Station et Event. Une station $i$ contient un nombre de places, une intensité d'arrivées de clients, ainsi qu'une liste de probabilités de trajet de la station $i$ aux autres stations. Les events sont les trajets entre stations et les arrivées de nouveaux clients aux stations. Ils sont stockés dans une queue de priorités en fonction du moment auquel ils vont se produire. Ils contiennent un temps et une station d'arrivée.

In [13]:
class Station:
    
    def __init__(self, _id, _nb_places, _client_intensity, _initial_fullness, _probabilities_of_target_station):
        self.id = _id
        self.nb_places = _nb_places
        self.client_intensity = _client_intensity
        self.nb_used_places = _initial_fullness
        self.probabilities_of_target_station = _probabilities_of_target_station
        

class Event:
    
    def __init__(self, _name, _time, _station, _target_station = None):
        self.name = _name
        self.time = _time
        self.station = _station
        self.target_station = _target_station
    
    def __lt__(self,other):
        return (self.time < other.time)

    def __str__(self): #Overload print(Event e)
        if self.name == "Pedestrian arrival":
            return "Event Pedestrian : Time={0:.3f} - At S{1}".format(self.time, self.station.id)
        elif self.name == "Bike arrival":
            return "Event Bike       : Time={0:.3f} - From S{1} to S{2}".format(self.time, self.station.id, self.target_station.id)
    
    def handle(self, list_stations, mean_travel_times):
        new_events = []
        if self.name == "Pedestrian arrival":
            if self.station.nb_used_places > 0 :
                destination = np.random.choice(list_stations, p = self.station.probabilities_of_target_station)
                mtt = mean_travel_times[list_stations.index(self.station)][list_stations.index(destination)]
                new_events.append(Event("Bike arrival", self.time +  np.random.exponential(mtt), self.station, destination))
                self.station.nb_used_places -= 1
            else:
                print("-->station vide")
            new_events.append(Event("Pedestrian arrival", self.time +  np.random.exponential(self.station.client_intensity), self.station))
            
        
        if self.name == "Bike arrival":
            if self.station.nb_places > self.station.nb_used_places:
                self.target_station.nb_used_places += 1
            else:
                print("-->station pleine")
                destination = np.random.choice(list_stations, p = self.target_station.probabilities_of_target_station)
                mtt = mean_travel_times[list_stations.index(self.target_station)][list_stations.index(destination)]
                new_events.append(Event("Bike arrival", self.time +  np.random.exponential(mtt), self.target_station, destination))
        
        return new_events

#### 2 - Données en entrée

On utilise les données fournies dans le fichier <code>velib_data_simulation</code>.

In [14]:
nb_places = [24, 20, 20, 15, 20]
initial_fullness = [20, 15, 17, 13, 18]
client_intensities = [1/2.8, 1/3.7, 1/5.5, 1/3.5, 1/4.6]
list_probabilities_of_target_station = [[0,    0.2,  0.3,  0.2,  0.3 ],
                                        [0.2,  0,    0.3,  0.2,  0.3 ],
                                        [0.2,  0.25, 0,    0.25, 0.3 ],
                                        [0.15, 0.2,  0.3,  0,    0.35],
                                        [0.2,  0.25, 0.35, 0.2,  0   ]]
mean_travel_times = [[0, 3, 5, 7, 7],
                     [2, 0, 2, 5, 5],
                     [4, 2, 0, 3, 3],
                     [8, 6, 4, 0, 2],
                     [7, 7, 5, 2, 0]]
ending_time = 100

#### 3 - Simulation

On définit ensuite la fonction qui va simuler les trajets entre les stations. La queue des événements est initialisée avec une arrivée de clients à chacune des stations. Ces arrivées suivent une loi exponentielle de paramètre donné dans le tableau <code>time_of_next_arrival_to_station</code>. 

In [15]:
def simulation(nb_places, client_intensities, mean_travel_times, initial_fullness, ending_time, list_probabilities_of_target_station):
    list_events = []
    current_time = 0
    list_stations = []  
    for i in range(len(nb_places)):
        list_stations.append(Station(i+3, nb_places[i], client_intensities[i], initial_fullness[i], list_probabilities_of_target_station[i]))
    
    time_of_next_arrival_to_station = np.random.exponential(client_intensities)
    list_events = [Event("Pedestrian arrival", current_time + time_of_next_arrival_to_station[i], list_stations[i]) for i in range(len(list_stations))]
    hp.heapify(list_events)
    
    while not current_time > ending_time:
        event = hp.heappop(list_events)
        print(event)
        current_time = event.time
        list_new_events = event.handle(list_stations, mean_travel_times)
        for new_event in list_new_events:
            hp.heappush(list_events, new_event)
    print("Fin ")

LANCEMENT OUÉÉÉÉÉÉ

In [16]:
simulation(nb_places, client_intensities, mean_travel_times, initial_fullness, ending_time, list_probabilities_of_target_station)

Event Pedestrian : Time=0.063 - At S7
Event Pedestrian : Time=0.078 - At S7
Event Pedestrian : Time=0.101 - At S7
Event Pedestrian : Time=0.104 - At S6
Event Pedestrian : Time=0.199 - At S7
Event Pedestrian : Time=0.201 - At S6
Event Pedestrian : Time=0.213 - At S5
Event Pedestrian : Time=0.246 - At S4
Event Pedestrian : Time=0.257 - At S6
Event Pedestrian : Time=0.266 - At S5
Event Pedestrian : Time=0.354 - At S6
Event Pedestrian : Time=0.391 - At S5
Event Pedestrian : Time=0.391 - At S4
Event Pedestrian : Time=0.417 - At S3
Event Bike       : Time=0.501 - From S5 to S7
Event Pedestrian : Time=0.533 - At S6
Event Pedestrian : Time=0.544 - At S6
Event Pedestrian : Time=0.565 - At S4
Event Pedestrian : Time=0.636 - At S7
Event Bike       : Time=0.693 - From S6 to S3
Event Pedestrian : Time=0.839 - At S6
Event Pedestrian : Time=0.872 - At S3
Event Pedestrian : Time=1.054 - At S4
Event Pedestrian : Time=1.067 - At S5
Event Pedestrian : Time=1.133 - At S6
Event Pedestrian : Time=1.144 - At